### Imports and Global Declarations

In [2]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
from wcs_client_adapter import WcsClientAdapter
from wcs_client_adapter import COLLECTION_TEXT_KEY, WCS_COLLECTION_NAME

def display_md(content):
  display(Markdown(content))

### Configure Language Model and Retrieval Model

In [3]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo")

wcs_client = WcsClientAdapter.get_wcs_client()
wcs_rm = WeaviateRM(WCS_COLLECTION_NAME, weaviate_client=wcs_client, weaviate_collection_text_key=COLLECTION_TEXT_KEY)
dspy.settings.configure(lm=turbo, rm=wcs_rm)

### Load Dataset

### Build Signatures

### Build Rag Pipeline as DSPy Module

### Optimize Pipeline

### Execute Pipeline

### Evaluate Complete RAG Pipeline

### Evaluate Retrieval Pipeline Stage